In [2]:
import numpy as np
import numpy.linalg as LA
from scipy import integrate
import matplotlib.pyplot as plt

## Метод колокації

Маємо інтегральне рівняння вигляду

$$
\dots
$$

Наближений розв'язок будемо шукати у вигляді

 $$
 \tilde{\psi_k}(x)=\sum_{j=1}^{n} c^{(k)}_{j} \gamma^{(k)}_{j}(x), \quad k = 1,2
 $$

де $c_j$ - невідомі константи, які потрібно знайти, а $\gamma_j$ - задані лінійнонезалежні функції (в нашому випадку кусково аналітичні), $j = 1, ..., n$.

Розглянувши його на множині точок $x_i$, таких, що
$$
a \leqslant x_{1}<x_{2}<\ldots<x_{m} \leqslant b
$$

отримаємо СЛАР відносно невідомих $с^{(k)}_j$, $j = 1, ..., n$. 

 $$
 A =
 \begin{pmatrix}
	 \begin{matrix}
	 	G^{(1)}_{11} & \dots  & G^{(1)}_{1n} \\
	 	\vdots 		 & \ddots & \\
	 	G^{(1)}_{n1} & 		  & G^{(1)}_{nn} \\
	 \end{matrix} &
	 \begin{matrix}
	 	G^{(2)}_{11} & \dots  & G^{(2)}_{1n} \\
	 	\vdots 		 & \ddots & \\
	 	G^{(2)}_{n1} & 		  & G^{(2)}_{nn} \\
	 \end{matrix} \\
	 \begin{matrix}
		G^{(3)}_{11} & \dots  & G^{(3)}_{1n} \\
		\vdots 		 & \ddots & \\
		G^{(3)}_{n1} & 		  & G^{(3)}_{nn} \\
	 \end{matrix} &
	 \begin{matrix}
		G^{(4)}_{11} & \dots  & G^{(4)}_{1n} \\
		\vdots 		 & \ddots & \\
		G^{(4)}_{n1} & 		  & G^{(4)}_{nn} \\
	 \end{matrix} \\
 \end{pmatrix}
 c = 
 \begin{pmatrix}
	c^{(1)}_1\\
	\vdots\\
	c^{(1)}_n\\
	c^{(2)}_1\\
	\vdots\\
	c^{(2)}_n\\
 \end{pmatrix}
 g = 
 \begin{pmatrix}
	2\pi g_1(x_1)\\
	\vdots\\
	2\pi g_1(x_n)\\
	2\pi g_2(x_1)\\
	\vdots\\
	2\pi g_2(x_n)\\
 \end{pmatrix}
 $$

In [38]:
a, b = 0, 0

x1 = lambda t: (np.cos(t), np.sin(t))
x2 = lambda t: (2*np.cos(t), 2*np.sin(t))
dx1 = lambda t: (-np.sin(t), np.cos(t))
dx2 = lambda t: (-2*np.sin(t), 2*np.cos(t))
ddx2 = lambda t: (-2*np.cos(t), -2*np.sin(t))

g1 = lambda x: 1
g2 = lambda x: 0

u = lambda x, y : 1

K11 = lambda t, tau: np.log(1/(LA.norm(np.substract(x1(t), x1(tau))))) \
                       if not np.isclose(t, tau) else \
                         - 1/2*np.log(4/np.e * np.sin((t - tau)/2)**2) \
                         + np.log(1/(np.e * LA.norm(dx1(t))**2))

K12 = lambda t, tau: np.log(1/(LA.norm(np.subtract(x1(t), x2(tau)))))

K21 = lambda t, tau: np.dot(np.subtract(x2(t), x1(tau)),
                            (dx1(tau)[1]/LA.norm(dx1(tau)), - dx1(tau)[0]/LA.norm(dx1(tau))))

K22 = lambda t, tau: dx2(t)[1]*(x2(t)[0] - x2(tau)[0]) - dx2(tau)[0]*(x2(t)[1] - x2(tau)[1]) \
                                        / LA.norm(x1(t) - x1(tau)) \
                       if not np.isclose(t, tau) else \
                         ddx2(t)[0]*dx2(t)[1] - ddx2(t)[1]*dx2(t)[0] \
                                    / 2*LA.norm(dx2(t))**2

h = 0.1

In [39]:
xx = np.arange(a, b + 0.000001, h).round(3).tolist()
n = len(xx)

In [40]:
def basicFunction(j, x, xx=xx, h=h):
  n = len(xx)-1 ## TODO
  
  if j >=1 and xx[j-1] <= x <= xx[j]:
    return (x - xx[j-1])/h
  elif j <= n-1 and xx[j] <= x <= xx[j+1]:
    return (xx[j+1] - x)/h
  else:
    return 0

$$
G^{(1)}_{ji} = \int_{0}^{2\pi} \gamma^{(1)}_{j}(\tau) K_{11}(t_i, \tau) d \tau
$$

In [41]:
G1 = np.zeros((n,n)).tolist()
for i in range(n):
  for j in range(n):
    
    Kxi_li= lambda s: K11(xx[i], s) * basicFunction(j, s)
    
    (integral, err) = integrate.quad(Kxi_li, a, b)
    
    G1[i][j] = integral

<ipython-input-38-bcedee8ca719>:16: RuntimeWarning: divide by zero encountered in log
  - 1/2*np.log(4/np.e * np.sin((t - tau)/2)**2) \
<ipython-input-41-c80ec9ae43ae>:5: RuntimeWarning: invalid value encountered in double_scalars
  Kxi_li= lambda s: K11(xx[i], s) * basicFunction(j, s)
<ipython-input-41-c80ec9ae43ae>:7: IntegrationWarning: Extremely bad integrand behavior occurs at some points of the
  integration interval.
  (integral, err) = integrate.quad(Kxi_li, a, b)


$$
G^{(2)}_{ji} = \int_{0}^{2\pi} \gamma^{(2)}_{j}(\tau) K_{12}(t_i, \tau) d \tau
$$

In [42]:
G2 = np.zeros((n,n)).tolist()
for i in range(n):
  for j in range(n):
    
    Kxi_li= lambda s: K12(xx[i], s) * basicFunction(j, s)
    
    (integral, err) = integrate.quad(Kxi_li, a, b)
    
    G2[i][j] = integral

$$
G^{(3)}_{ji} = \int_{0}^{2\pi} \gamma^{(1)}_{j}(\tau) K_{21}(t_i, \tau) d \tau
$$

In [43]:
G3 = np.zeros((n,n)).tolist()
for i in range(n):
  for j in range(n):
    
    Kxi_li= lambda s: K21(xx[i], s) * basicFunction(j, s)
    
    (integral, err) = integrate.quad(Kxi_li, a, b)
    
    G3[i][j] = integral

$$
G^{(4)}_{ji} = -\pi\frac{\gamma^{(2)}_{j}(t_i)}{|x'_{2}(t_i))|}
				 + \int_{0}^{2\pi} \gamma^{(2)}_{j}(\tau) K_{22}(t_i, \tau) d \tau
$$

In [44]:
G4 = np.zeros((n,n)).tolist()
for i in range(n):
  for j in range(n):
    
    Kxi_li= lambda s: K22(xx[i], s) * basicFunction(j, s)
    
    (integral, err) = integrate.quad(Kxi_li, a, b)
    
    G4[i][j] = integral - np.pi * basicFunction(j, xx[i]) / LA.norm(dx2(xx[i]))

In [45]:
A = np.concatenate((np.concatenate((G1, G2), axis=1),
                    np.concatenate((G3, G4), axis=1)), axis=0)
A.shape

(2, 2)

In [46]:
g = np.concatenate(([[g1(xx[j])] for j in range(n)],
                    [[g2(xx[j])] for j in range(n)]))
g.shape

(2, 1)

In [48]:
A

array([[nan,  0.],
       [ 0., -0.]])

In [ ]:
g

In [47]:
LA.det(A)

c:\users\bobuh\appdata\local\programs\python\python38\lib\site-packages\numpy\linalg\linalg.py:2116: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


nan

In [ ]:
c = LA.solve(A, 2*np.pi*g)

In [ ]:
def psi1_approx(x, c=c):
  return sum(c[i][0]*basicFunction(i, x) for i in range(n))

def psi2_approx(x, c=c):
  return sum(c[n + i][0]*basicFunction(i, x) for i in range(n))

In [ ]:
K1 = lambda x, tau: np.log(1/LA.norm()) ???
K2 = lambda x, tau:

In [ ]:
u_approx = lambda x: 1/(2*np.pi) * integrate.quad(psi1_approx(x psi1_approx(x, a, b)

In [ ]:
# TODO перевірити базис Гамма